In [1]:
import os
import sys

import requests
from flask import Flask, request
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
# things we need for Tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import adam,SGD
import tensorflow as tf
import re
import pandas as pd
import json
import pickle
import random

# load dataset from json file which is given in the folder
with open('try-copy1.json',encoding='utf-8') as json_data:
    intents = json.load(json_data)
context = {}

words = []
classes = []
documents = []
ignore_words =['?','ተብሎ','የሚጠራ','ሲሆን','ይህም','ወይም','በሌሎች','ላይ','ነው','በ','የ','ከ','ሊሆኑ','ማለት','ይችላሉ','ወይ','ይችላል','በጣም','ከመሆን','የተነሳ','ወደ','ሙሉ','አይነት','ጨምሮ','የአንድ','አንድ','ግለሰብ','የሆነ','ሁኔታ','ውስጥ','ናቸው','ማለትም','የሚገኙ','በአደገኛ','አደገኛ','ወይንም','በአንድ','እንዲሁም','ሲሆን','በግዜው','ጨምሮ','ያሉ','ሰዎች','በአብዛኛውን','ሰው','ምንም','ሆኖ','ከሰው','ወደ','ከባድ','ምንድ','ምንድን','ምንድነው','ምንድናቸው','ነው','ናቸው','ስንል','ምን','ማለታችን','ማለት','አለው','የምን','የያዘው','እንዴት','የያዘውን','ሊሆን','የምን','በምን','የሚችል','ማን','በተለየ','በአብዛኛው','የቱን','የትኛው','እንዴት','አይነቶች','አለብን','ያለብን','ከመያዛችን','በፊት','የመያዝ','አጋጣሚን','አጋጣሚ','እንዳለብን','አሉት','በሰአቱ','በግዜው','ሁሉ','ሁሉም','ሆነ','ሆኖም','ሁሉንም','ማለት','ማን','ብቻ','ነገር','ነገሮች','ናቸው','አሁን','አለ','እስከ','እንኳን','እስከ','እዚሁ','እና','እንደ','ከ','ወዘተ','ወይም','ዋና','ይህ','ደግሞ','ጋራ','ግን','ጋር','ሆኖም','ማን','ለማን','ማነው','ማንማን','ማንን','ከማንኛው','ማንኛው','በማን','ጥቀስ','ግለፅ','ዘርዝር','ጥራ','ምን','ምንድን','የምን','ለምን','በምን','ወይ','ይሆን','እንደ','እንዴት']
def normalize(sentences):
    text=[i.replace('ኀ','ሀ').replace('ሐ','ሀ').replace('ሃ','ሀ').replace('ኃ','ሀ').replace('ሓ','ሀ').replace('ኁ','ሁ').replace('ሑ','ሁ').replace('ሒ','ሂ').replace('ኂ','ሂ').replace('ኄ','ሄ').replace('ሔ','ሄ').replace('ሕ','ህ').replace('ኅ','ህ').replace('ሖ','ሆ').replace('ኆ','ሆ').replace('ጸ','ፀ').replace('ጹ','ፁ').replace('ጺ','ፂ').replace('ጻ','ፃ').replace('ጼ','ፄ').replace('ጽ','ፅ').replace('ጾ','ፆ').replace('ቸ,','ቼ').replace('ሸ','ሼ').replace('ዬ','የ').replace('ዉ','ው').replace('ሓ','ሀ').replace('ሠ','ሰ').replace('ሡ','ሱ').replace('ሢ','ሲ').replace('ሣ','ሳ').replace('ሤ','ሴ').replace('ሥ','ስ').replace('ሦ','ሶ').replace('ዐ','አ').replace('ዑ','ኡ').replace('ዒ','ኢ').replace('ዓ','አ').replace('ኣ','አ').replace('ዔ','ኤ').replace('ዕ','እ').replace('ዖ','ኦ').replace('መካኒካል','ሜካኒካል').replace('ኢንጂነሪንግ','ምህንድስና').replace('ሰፍትዌር','ሶፍትዌር').replace('ሲስተም','ስይስተም').replace('ከሚካል','ኬሚካል').replace('ደምወዝ','ደሞዝ').replace('ዶሞዝ','ደሞዝ').replace('አርኪተክቸር','አርክተክቸር').replace('ኢሌክትሪካል','ኤሌክትሪካል').replace('ኮምፒተር','ኮምፕዩተር').replace('ሳይነስ','ሳይንስ').replace('ኢንፎርመሽን','ኢንፎርሜሽን').replace('ኢንዱስትርያል','ኢንዳስትሪያል').replace('ኢንዱስትሪያል','ኢንዳስትሪያል').replace('መሃንዲስ','መሀንድስ') for i in sentences]    
    return(text)

# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words= normalize(words)
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)


ModuleNotFoundError: No module named 'flask'

In [58]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
x = list(training[:,0])
y = list(training[:,1])

from sklearn.model_selection import train_test_split as tts
train_x,test_x,train_y,test_y=tts(x, y, test_size=.1, random_state=42)
print("the 80% of the dataset for training the model is:",len(train_x))
print("the 20% of test_data for validation is :",len(test_y))

val_x=np.array(test_x)
val_y=np.array(test_y)

the 80% of the dataset for training the model is: 1624
the 20% of test_data for validation is : 181


In [59]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(64, ))
model.add(Dense(32, ))
#model.add(Dropout(0.2))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
Adam = adam(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=Adam, metrics=['accuracy'])

# simple early stopping
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=20)
# Fit the model
history=model.fit(np.array(x), np.array(y), epochs=150, batch_size=8,validation_data=(val_x,val_y),  verbose=1,callbacks=[es])


Train on 1805 samples, validate on 181 samples
Epoch 1/150
1805/1805 [==============================] - 2s 1ms/step - loss: 4.8066 - acc: 0.0659 - val_loss: 4.7603 - val_acc: 0.1271
Epoch 2/150
1805/1805 [==============================] - 1s 664us/step - loss: 4.6817 - acc: 0.1285 - val_loss: 4.5113 - val_acc: 0.1271
Epoch 3/150
1805/1805 [==============================] - 1s 546us/step - loss: 4.3297 - acc: 0.1141 - val_loss: 4.1059 - val_acc: 0.1326
Epoch 4/150
1805/1805 [==============================] - 1s 545us/step - loss: 4.1084 - acc: 0.1119 - val_loss: 3.9767 - val_acc: 0.1326
Epoch 5/150
1805/1805 [==============================] - 1s 545us/step - loss: 3.9951 - acc: 0.1180 - val_loss: 3.8671 - val_acc: 0.1381
Epoch 6/150
1805/1805 [==============================] - 1s 546us/step - loss: 3.8836 - acc: 0.1269 - val_loss: 3.7500 - val_acc: 0.1602
Epoch 7/150
1805/1805 [==============================] - 1s 574us/step - loss: 3.7571 - acc: 0.1535 - val_loss: 3.6146 - val_acc: 0.1

1805/1805 [==============================] - 1s 569us/step - loss: 0.0541 - acc: 0.9690 - val_loss: 0.0487 - val_acc: 0.9724
Epoch 120/150
1805/1805 [==============================] - 1s 575us/step - loss: 0.0544 - acc: 0.9706 - val_loss: 0.0505 - val_acc: 0.9669
Epoch 121/150
1805/1805 [==============================] - 1s 568us/step - loss: 0.0536 - acc: 0.9684 - val_loss: 0.0496 - val_acc: 0.9613
Epoch 122/150
1805/1805 [==============================] - 1s 565us/step - loss: 0.0544 - acc: 0.9701 - val_loss: 0.0457 - val_acc: 0.9834
Epoch 123/150
1805/1805 [==============================] - 1s 578us/step - loss: 0.0538 - acc: 0.9695 - val_loss: 0.0502 - val_acc: 0.9779
Epoch 124/150
1805/1805 [==============================] - 1s 570us/step - loss: 0.0542 - acc: 0.9684 - val_loss: 0.0487 - val_acc: 0.9669
Epoch 125/150
1805/1805 [==============================] - 1s 574us/step - loss: 0.0537 - acc: 0.9679 - val_loss: 0.0480 - val_acc: 0.9724
Epoch 126/150
1805/1805 [================

In [ ]:
import matplotlib.pyplot as plt
loss_train = history.history['acc']
loss_val = history.history['val_acc']
epochs = range(1, len(loss_train) + 1)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(1, len(loss_train) + 1)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [61]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
pred = model.predict(val_x)
predicted=np.argmax(pred, axis=1)
report=classification_report(np.argmax(val_y, axis=1),predicted)

result = model.evaluate(val_x,val_y, verbose=0)
#res=np.argmax(result, axis=1)
f1=f1_score(np.argmax(val_y, axis=1),predicted,average="macro")
precision=precision_score(np.argmax(val_y, axis=1),predicted,average="macro")
recall=recall_score(np.argmax(val_y, axis=1),predicted,average="macro")

print("Accuracy: %.2f%%" % (result[1]*100))

#y_pred = model.predict(testx)
print("F1 Score: %.2f%%" % (f1*100))
print("Precision Score: %.2f%%" % (precision*100)) 
print("Recall Score: %.2f%%" % (recall*100))

Accuracy: 97.24%
F1 Score: 96.12%
Precision Score: 96.51%
Recall Score: 96.32%


In [62]:
model.save('model_FAQ_best_wzz_english_ChatBot.h5')

In [34]:
model.save('model_FAQ_best_ChatBot.h5')

In [2]:
# save all of our data structures
from keras.models import load_model
#model=load_model('model_FAQ_best_ChatBot.h5')
model=load_model('model_FAQ_best_wzz_english_ChatBot.h5')
def clean_up_sentence(sentence):
    #sentence_words= normalize(sentence)
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words if word not in ignore_words]
    sentence_words= normalize(sentence_words)
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    p = bow(sentence, words)
    
    d = len(p)
    f = len(documents)-2
    a = np.zeros([f, d])
    tot = np.vstack((p,a))
    
    results = model.predict(tot)[0]
    
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list
not_answerd=[]
from collections import Counter
import collections
def response(sentence, userID='12', show_details=False):
    results = classify(sentence)
    #print('Result:',results)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return (random.choice(i['responses']))
            results.pop(0)
    else:
        not_answerd.append(sentence)
        c = Counter(not_answerd[1:])  # slice to ignore first value, Counter IS a dict already 
        # Just output counts > 3
        with open('listfile.txt', 'w', encoding='utf-8') as filehandle:
            for k, v in c.items():
                if v > 2:
                    filehandle.write('%s\n' % k)
                    #print(k)
        res="አልገባኝም አስተካክለህ በድጋሜ ጠይቅኝ።"
        return res  
       #print (not_answerd)

In [29]:
from gtts import gTTS
import os
from pygame import mixer

def speak(T):
    mytext=T
    language='en'
    myobj=gTTS(text=mytext,lang=language,slow=False)
    myobj.save("speak.mp3")
    mixer.init()
    mixer.music.load('speak.mp3')
    return mixer.music.play()

import speech_recognition as sr
import time

def listen():
    r=sr.Recognizer()
    speak('i am listening')
    time.sleep(2)
    with sr.Microphone() as source:
        print(emoji.emojize('\n\n:information_desk_person:: i am Listening',use_aliases=True))
        audio=r.listen(source,timeout=5,phrase_time_limit=15)
        try:
            text=r.recognize_google(audio)
            print(emoji.emojize('\n\n:man::',use_aliases=True),"{}".format(text))
        except:
            print(emoji.emojize('\n\n:information_desk_person:: i am listening',use_aliases=True),
                  'sorry could not hear u')
            speak("sorry could not hear u")
            time.sleep(5)
            print(emoji.emojize('\n\n:information_desk_person::',use_aliases=True),
                 'please repeat it again')
            speak("please repeat it again")
            audio=r.listen(source,timeout=5,phrase_time_limit=15)
            try:
                text=r.recognize_google(audio)
                print(emoji.emojize('\n\n:man::',use_aliases=True),"{}".format(text))
            except:
                text="quit"
        return text

In [19]:
def voicechat():
    speak("i am FAQ")
    print(emoji.emojize('\n i am FAQ:information_desk_person:',use_aliases=True))
    time.sleep(3)
    while True:
        inp=listen()
        if inp=="quit":
            print(emoji.emojize('\n\n:information_desk_person::',use_aliases=True),
                  '10q bye')
            speak("\n10q bye")
            break
        
        answer=response(inp)
        speak(answer)
        print(emoji.emojize('\n\n:information_desk_person::',use_aliases=True),answer)
        length=len(answer)
        time.sleep(5)

In [22]:
import pyttsx3
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)
volume = engine.getProperty('volume')
engine.setProperty('volume', 10.0)
rate = engine.getProperty('rate')
engine.setProperty('rate', rate - 25)

In [38]:
import gtts
from playsound import playsound
print(gtts.lang.tts_langs())

{'af': 'Afrikaans', 'sq': 'Albanian', 'ar': 'Arabic', 'hy': 'Armenian', 'bn': 'Bengali', 'bs': 'Bosnian', 'ca': 'Catalan', 'hr': 'Croatian', 'cs': 'Czech', 'da': 'Danish', 'nl': 'Dutch', 'en': 'English', 'eo': 'Esperanto', 'et': 'Estonian', 'tl': 'Filipino', 'fi': 'Finnish', 'fr': 'French', 'de': 'German', 'el': 'Greek', 'gu': 'Gujarati', 'hi': 'Hindi', 'hu': 'Hungarian', 'is': 'Icelandic', 'id': 'Indonesian', 'it': 'Italian', 'ja': 'Japanese', 'jw': 'Javanese', 'kn': 'Kannada', 'km': 'Khmer', 'ko': 'Korean', 'la': 'Latin', 'lv': 'Latvian', 'mk': 'Macedonian', 'ml': 'Malayalam', 'mr': 'Marathi', 'my': 'Myanmar (Burmese)', 'ne': 'Nepali', 'no': 'Norwegian', 'pl': 'Polish', 'pt': 'Portuguese', 'ro': 'Romanian', 'ru': 'Russian', 'sr': 'Serbian', 'si': 'Sinhala', 'sk': 'Slovak', 'es': 'Spanish', 'su': 'Sundanese', 'sw': 'Swahili', 'sv': 'Swedish', 'ta': 'Tamil', 'te': 'Telugu', 'th': 'Thai', 'tr': 'Turkish', 'uk': 'Ukrainian', 'ur': 'Urdu', 'vi': 'Vietnamese', 'cy': 'Welsh', 'zh-cn': 'Chin

In [ ]:
while True:
    #now = datetime.datetime.now()
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Tell me something:")
        engine.say('ሰላም')
        engine.runAndWait()
        audio = r.listen(source)
        try:
            print("You said:- " + r.recognize_google(audio))
            engine.say('I am fine')
            engine.runAndWait()
        except sr.UnknownValueError:
            print("Could not understand audio")
            engine.say('I didnt get that. Rerun the code')
            engine.runAndWait()

In [23]:
response('ፈተና አለመፈተን', '123', show_details=True)

found in bag: ፈተና
found in bag: አለመፈተን
tag: ፈተና ዋጋ


'አሳይመንቶች እና አጫጭር ፈተናዎች እንደ አስተማሪው አሰጣጥ ሁኖ ጠቅላላ ከ 50% ይታሰባል፤ የመጨረሻ ፈተና ደሞ ከ 50% ይታሰባል።'

In [64]:
while 1:
    question=input("you: ")
    if question=='':
        print("FAQ Bot: ይቅርታ ምንም አልፃፍክም!")
    if question=='bye':
        print("FAQ Bot: ", response(question))
        break
    elif question=="ቻው":
        print("FAQ Bot: ቻው መልካም ጊዜ!")
        break
    else:
        print("FAQ Bot: ", response(question))

you: ዲፓርትመንቶች
found in bag: ዲፓርትመንቶች
FAQ Bot:  የመቐለ ዩኒቨርሲቲ EIT-M የአካዳሚ የመጀመሪያ ድግሪ መርሃግብሮች 
1. አርክቴክቸር 
2. አርባን ፕላኒንግና ዲዛይን 
3. ሲቪል ኢንጂነሪንግ 
4. ወተር ሪሶርስና ኢሪጌሽን ኢንጂነሪንግ 
5. ኤሌክትሪካልና ኮምፒተር ኢንጂነሪንግ 
6. መካኒካል ኢንጂነሪንግ 
7.ኢንዳስትሪያል ኢንጂነሪንግ 
8.ኬሚካል ኢንጂነሪንግ 
9.ኮምፒተር ሳይንስ 
10.ኢንፎርሜሽን ስይስተም እናም 
11. የሶፍትዌር ኢንጂነሪንግ ናቸው፡፡ ስለ የቱን ምህንድስና ዲፓርትመንት ማወቅ ትፈልጋላችሁ?
you: ወተር ሪሶርስና ኢሪጌሽን ኢንጂነሪንግ
found in bag: ወተር
found in bag: ምህንድስና
FAQ Bot:  የውሃ ሀብቶች ምህንድስና(ወተር ሪሶርስ ምህንድስና) ለሰው ልጆች የውሃ አቅርቦትን እና ለደህንነት ሲባል የቆሸሸ ውሃ መወገድን የሚመለከቱ መዋቅሮች እና ሂደቶችን የሚመለከት የምህንድስና መስክ ነው ፡፡ የውሃ ሀብቶች ምህንድስና(ወተር ሪሶርስ ምህንድስና) በውኃ አቅርቦት ፣ በመስኖ እና በቆሻሻ ማስወገጃ ላይ የሚያተኩር የሲቪል ምህንድስና ልዩ መስክ ነው ፡፡ እንዲሁም ከውኃ ጋር የተዛመዱ ጉዳቶችን እና አደጋዎችን ለማስወገድ ውሃን ለመቆጣጠር የሚረዱ ዘዴዎችን ያብራራል።
you: ስለ ኣርባን ፕላኒንግ
found in bag: ስለ
found in bag: አርባን
FAQ Bot:  ያምራል፣ ነገ የበለጠ እንነጋገራለን፡፡
you: ስለ አርባን ፕላኒንግና ዲዛይን
found in bag: ስለ
found in bag: አርባን
found in bag: ፕላኒንግና
found in bag: ዲዛይን
FAQ Bot:   የከተማ ፕላን (አርባን ፕላኒንግና ዲዛይን) ምህንድስና የከባብያዊ አየር፣ ውሃን እና እንደ የከተማ መጓጓዣ እና ማሰራጫ አው